In [1]:
import pandas as pd
import duckdb

In [2]:
with duckdb.connect('../data/RE_moscow_db.duckdb') as w:
    for table in w.execute('PRAGMA show_tables').df()['name'].values:
        print(table,' :',w.execute(f'select count(*) as events from {table}').df().iloc[0].values[0])

agency_ads  : 8143
builder_ads  : 8786
home_info  : 32933
user_ads  : 6393


In [3]:
with duckdb.connect('../data/general_moscow_home_info.duckdb') as w:
    for table in w.execute('PRAGMA show_tables').df()['name'].values:
        print(table,' :',w.execute(f'select count(*) as events from {table}').df().iloc[0].values[0])

home_base_info  : 32933


In [3]:
with duckdb.connect('../data/RE_moscow_db.duckdb') as w:
    agency_df = w.execute(f'select * from agency_ads').df().assign(ads_type='agency_ads')
    builder_df = w.execute(f'select * from builder_ads').df().assign(ads_type='builder_ads')
    user_df = w.execute(f'select * from user_ads').df().assign(ads_type='user_ads')

In [5]:
agency_df.groupby(pd.qcut(agency_df['m2'],3))['price'].describe().div(1000).round(3).style.format('{:_}')

,count,mean,std,min,25%,50%,75%,max
m2,,,,,,,,
"(11.999, 57.0]",2.825,15_532.181,6_425.931,4_856.2,11_350.0,13_800.0,17_950.0,71_500.0
"(57.0, 77.0]",2.671,24_649.562,15_880.79,14.5,15_951.28,21_400.0,29_000.0,200_000.0
"(77.0, 272.0]",2.647,63_437.897,55_750.193,6_654.23,29_900.0,44_500.0,73_000.0,377_328.328


In [4]:
full_df = pd.concat([agency_df,builder_df,user_df])

In [19]:
sanya = ['Печатники', 'Люблино', 'Алтуфьево', 'Бибирево']

In [27]:
full_df.query('metro in @sanya').to_csv('sanya_avito.csv',index=False)

In [7]:
full_df['datetime'].describe()

/tmp/ipykernel_485/1008442374.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  full_df['datetime'].describe()


count                                23322
unique                               23322
top       2022-10-30 21:32:09.995889+00:00
freq                                     1
first     2022-10-30 20:57:46.860878+00:00
last      2022-12-02 16:41:24.215882+00:00
Name: datetime, dtype: object